### 8.1 코너점 검출

In [1]:
import cv2
import numpy as np

def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))
    
    dilate = cv2.dilate(src, kernel)
    localMax = (src == dilate)
    
    erode = cv2.erode(src, kernel)
    localMax2 = src > erode
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:, [0, 1]] = points[:, [1, 0]]
    return points

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize = 3)
ret, res2 = cv2.threshold(np.abs(res), 0.1, 0, cv2.THRESH_TOZERO)
corners = findLocalMaxima(res2)
print(corners.shape)

dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 5, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(8, 2)


### 8.2 고유값, 고유벡터를 이용한 검출

In [3]:
import cv2
import numpy as np

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerEigenValsAndVecs(gray, 5, 3)
print(res.shape)
eigen = cv2.split(res)

T = 0.2
ret, edge = cv2.threshold(eigen[0], T, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

corners = np.argwhere(eigen[1] > T)
corners[:, [0, 1]] = corners[:, [1, 0]]
print(len(corners))
print(corners) # corner 좌표

dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 5, (0, 0, 255), 2)

cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(512, 512, 6)
8
[[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]


### 8.3 최소 고유값을 통한 검출

In [8]:
import cv2
import numpy as np

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
eigen = cv2.cornerMinEigenVal(gray, 5)
print(eigen.shape)

T = 0.2
corners = np.argwhere(eigen>T)
corners[:, [0, 1]] = corners[:, [1, 0]]
print(len(corners))
print(corners)
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 3, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(512, 512)
8
[[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]


### 8.4 일반적으로 사용하는 코너 점 검출

In [13]:
import cv2
import numpy as np

def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))
    
    dilate = cv2.dilate(src, kernel)
    localMax = (src == dilate)
    
    erode = cv2.erode(src, kernel)
    localMax2 = src > erode
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:, [0, 1]] = points[:, [1, 0]]
    return points

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, 5, 3, 0.01)
ret, res = cv2.threshold(np.abs(res), 0.02, 0, cv2.THRESH_TOZERO)
res8 = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imshow('res8', res8)

corners = findLocalMaxima(res)
print(len(corners), corners)

# 좀 더 정교한 corner 찾기
corners = corners.astype(np.float32, 'C')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), term_crit)
print(len(corners2), corners2)


dst = src.copy()
for x, y in np.int32(corners2):
    cv2.circle(dst, (x, y), 5, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

8 [[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]
8 [[107.559364 125.559456]
 [265.44077  125.559235]
 [265.55923  168.44174 ]
 [387.4408   168.55888 ]
 [107.558365 269.44095 ]
 [168.4415   269.55927 ]
 [168.55922  375.4408  ]
 [387.4408   375.4408  ]]


### 8.6 비디오에서 사용 가능한 corner 검출

In [2]:
import cv2
import numpy as np

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

K = 5 # 검출할 corner의 개수
corners = cv2.goodFeaturesToTrack(gray, K, 0.05, 10)
print(corners.shape, len(corners), corners)

corners2 = cv2.goodFeaturesToTrack(gray, K, 0.05, 10, useHarrisDetector=True, k = 0.04)
print(corners2.shape, len(corners2), corners)

dst = src.copy()
corners = corners.reshape(-1, 2)
for x, y in corners:
    cv2.circle(dst, (int(x), int(y)), 5, (0, 0, 255), -1)

corners2 = corners2.reshape(-1, 2)
for x, y in corners2:
    cv2.circle(dst, (int(x), int(y)), 5, (255, 0, 0), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(5, 1, 2) 5 [[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[168. 270.]]

 [[266. 168.]]]
(5, 1, 2) 5 [[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[168. 270.]]

 [[266. 168.]]]


### 8.7 체스보드 패턴 검출

In [15]:
import cv2
import numpy as np

# 패턴 사이즈가 원본과 일치해야 오류가 안생김

# 7x7
# src = cv2.imread('c:/data/chess.jpg')
# patternSize = (7, 7)
src = cv2.imread('c:/data/chessBoard.jpg')
patternSize = (6, 3)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
found, corners = cv2.findChessboardCorners(src, patternSize)
print(corners.shape, len(corners), corners)

term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), term_crit)

dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, corners2, found)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(18, 1, 2) 18 [[[455.34433 226.53378]]

 [[409.39923 219.81999]]

 [[367.74002 215.4076 ]]

 [[324.20044 213.23212]]

 [[278.29123 209.68956]]

 [[238.23654 205.51662]]

 [[460.6499  178.78876]]

 [[414.6205  176.80276]]

 [[370.02316 170.8178 ]]

 [[326.50217 169.51678]]

 [[286.254   165.41183]]

 [[243.83284 159.94035]]

 [[468.24713 132.65489]]

 [[419.30606 128.9939 ]]

 [[378.1978  126.20462]]

 [[334.00836 124.61804]]

 [[291.51782 118.95656]]

 [[246.54684 116.65251]]]


### 8.8 원 패턴 중심점 검출

In [14]:
import cv2
import numpy as np

src = cv2.imread('c:/data/circleGrid.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
patternSize = (6, 4)
found, centers = cv2.findCirclesGrid(src, patternSize)
print(centers.shape, len(centers), centers)

term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
centers2 = cv2.cornerSubPix(gray, centers, (5, 5), (-1, -1), term_crit)

dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, centers2, found)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(20, 1, 2) 20 [[[ 49.494728  49.367813]]

 [[149.50102   49.364902]]

 [[249.50528   49.367813]]

 [[349.49472   49.367813]]

 [[449.50104   49.364902]]

 [[ 49.498344 149.11923 ]]

 [[149.50214  149.1125  ]]

 [[249.50272  149.11885 ]]

 [[349.49835  149.11923 ]]

 [[449.50214  149.1125  ]]

 [[ 49.498344 248.88077 ]]

 [[149.50214  248.8875  ]]

 [[249.50272  248.88115 ]]

 [[349.49835  248.88077 ]]

 [[449.50214  248.8875  ]]

 [[ 49.494728 348.6322  ]]

 [[149.50102  348.6351  ]]

 [[249.50528  348.6322  ]]

 [[349.49472  348.6322  ]]

 [[449.50104  348.6351  ]]]
